# LAB GenAI - LLMs

# OpenAI GPT API Exercises

## 1. Basic Conversation
**Exercise:** Create a simple chatbot that can answer basic questions about a given topic (e.g., history, technology).  
**Parameters to explore:** `temperature`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `n`, `stop`.

Comment what happen when you change the parameters 
(read documentation!)

In [40]:
import openai
from openai import OpenAI

# Initialize the OpenAI API key
openai.api_key = ''

In [24]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

def history_teacher_prompt(string, temperature=0.7, max_tokens=150, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, n=1, stop=None):
    messages = [
        {
            "role": "system",
            "content": "You need to answer like a history teacher would answer to a student, giving details and dates in a manner everyone can understand despite their lack of knowledge."
        },
        {
            "role": "user",
            "content": string
        }
    ]

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        n=n,
        stop=stop,
    )

    return chat_completion.choices[0].message.content

# Showcase different requests with varied parameter values
    # Default parameters

response1 = history_teacher_prompt('Who is Napoleon and what did he achieve?')
print("Response with default parameters:\n", response1)

# High temperature (more creative)
response2 = history_teacher_prompt('Who is Napoleon and what did he achieve?', temperature=1.0)
print("\nResponse with high temperature:\n", response2)

# Low temperature (more deterministic)
response3 = history_teacher_prompt('Who is Napoleon and what did he achieve?', temperature=0.3)
print("\nResponse with low temperature:\n", response3)

# Shorter response
response4 = history_teacher_prompt('Who is Napoleon and what did he achieve?', max_tokens=50)
print("\nResponse with fewer max tokens:\n", response4)

# Including stop sequence
response5 = history_teacher_prompt('Who is Napoleon and what did he achieve?', stop=["Napoleon"])
print("\nResponse with stop sequence:\n", response5)

# High frequency penalty (discourages repetition)
response6 = history_teacher_prompt('Who is Napoleon and what did he achieve?', frequency_penalty=1.0)
print("\nResponse with high frequency penalty:\n", response6)

# Presence penalty (encourages new topics)
response7 = history_teacher_prompt('Who is Napoleon and what did he achieve?', presence_penalty=1.0)
print("\nResponse with high presence penalty:\n", response7)


Response with default parameters:
 Napoleon Bonaparte was a French military general and political leader who rose to prominence during the French Revolution in the late 18th century. He became the Emperor of France in 1804 and went on to expand the French Empire across much of Europe through a series of military conquests known as the Napoleonic Wars.

Among his achievements, Napoleon centralized the French government, established the Napoleonic Code (a civil legal code that is still influential today), and implemented various reforms in education, infrastructure, and the economy. He also played a significant role in spreading the ideas of the French Revolution throughout Europe, including concepts such as nationalism and meritocracy.

However, Napoleon's ambitions eventually led to his downfall. He faced a series of military

Response with high temperature:
 Napoleon Bonaparte was a prominent military leader and emperor of France in the early 19th century. He rose to power during the 

In [23]:
# Call the function
history_teacher_prompt('Who is Napoleon and what did he achieve?')

"Napoleon Bonaparte was a French military general and emperor who rose to prominence during the French Revolution in the late 18th century. He eventually became the ruler of France and conquered much of continental Europe in the early 19th century.\n\nNapoleon's most notable achievements include instituting a series of legal reforms known as the Napoleonic Code, which influenced legal systems around the world. He also centralized the government, established a system of merit-based promotion for officials, and implemented public education reforms.\n\nIn terms of military conquests, Napoleon is known for his victories in battles such as Austerlitz, Jena, and Wagram. However, his military campaigns eventually led to his downfall, as he faced defeats in battles such as Leipzig and Waterloo.\n\nNapoleon was eventually exiled to the island of Saint Helena, where he died in 1821. Despite his eventual defeat, Napoleon's legacy continues to be the subject of historical debate, with some viewing

Temperature: more temperature increases the creativity, lower increases the determinism of the output.

Max Tokens: Limits the length of the generated response.

Top P: high = gives back the most likely tokens and makes the generation diverse, low = restricts to 10% of probable tokens, output becomes conservative and less diverse.

Frequency Penalty: Reduces repetition by penalizing word reuse as we increase it.

Presence Penalty: Encourages the introduction of new topics by penalizing repeated concepts as we increase it.

Number of Responses: Allows generating multiple responses based on a number for comparison.

Stop Sequence: Defines a point at which the response generation stops.


## 2. Summarization
**Exercise:** Write a script that takes a long text input and summarizes it into a few sentences.  
**Parameters to explore:** `temperature`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `best_of`, `logprobs`.

Comment what happen when you change the parameters 
(read documentation!)

In [28]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

def summarize_text(text, temperature=0.7, max_tokens=100, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, best_of=1, logprobs=None):
    messages = [
        {
            "role": "system",
            "content": "You need to summarize the given text into a few sentences in a manner that retains the key points and main ideas."
        },
        {
            "role": "user",
            "content": text
        }
    ]

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        logprobs=logprobs,
    )

    return chat_completion.choices[0].message.content

In [29]:
# Example usage in a Jupyter Notebook
text_to_summarize = """
Napoleon Bonaparte was a French military and political leader who rose to prominence during the French Revolution and led several successful campaigns during the Revolutionary Wars. He was Emperor of the French from 1804 until 1814, and again in 1815 during the Hundred Days. Napoleon dominated European and global affairs for more than a decade while leading France against a series of coalitions in the Napoleonic Wars. He won most of these wars and the vast majority of his battles, building a large empire that ruled over much of continental Europe before its final collapse in 1815. One of the greatest commanders in history, his wars and campaigns are studied at military academies worldwide. Napoleon's political and cultural legacy has endured as one of the most celebrated and controversial leaders in human history.
"""

summary = summarize_text(text_to_summarize)
print("Summary with default parameters:\n", summary)

Summary with default parameters:
 Napoleon Bonaparte was a prominent French military and political leader who played a significant role during the French Revolution and Revolutionary Wars. He became Emperor of the French in 1804 and led successful campaigns in the Napoleonic Wars, dominating European affairs for over a decade. Known for his military prowess, Napoleon's legacy as a commander and leader remains influential and controversial in history.


In [30]:
summary2 = summarize_text(text_to_summarize, temperature=1.0)
print("Summary with high temperature:\n", summary2)

Summary with high temperature:
 Napoleon Bonaparte was a prominent French military and political leader known for his successes during the French Revolution and Napoleonic Wars, serving as Emperor of France from 1804 to 1815. He dominated European affairs, leading France to victory in many battles and expanding its empire across continental Europe. As one of history's greatest commanders, his military strategies continue to be studied globally, and his legacy remains both celebrated and debated.


In [31]:
summary3 = summarize_text(text_to_summarize, temperature=0.3)
print("Summary with low temperature:\n", summary3)

Summary with low temperature:
 Napoleon Bonaparte was a prominent French military and political leader who played a significant role during the French Revolution and Revolutionary Wars. He became Emperor of the French in 1804 and led successful campaigns during the Napoleonic Wars, dominating European affairs for over a decade. Napoleon's legacy as a skilled commander and controversial figure continues to be studied and debated globally.


In [32]:
summary4 = summarize_text(text_to_summarize, max_tokens=50)
print("Summary with shorter max tokens:\n", summary4)

Summary with shorter max tokens:
 Napoleon Bonaparte was a prominent French military and political leader who played a significant role during the French Revolution and the Revolutionary Wars. As Emperor of the French from 1804 to 1815, he led successful campaigns in the Napole


In [33]:
summary5 = summarize_text(text_to_summarize, max_tokens=200)
print("Summary with longer max tokens:\n", summary5)

Summary with longer max tokens:
 Napoleon Bonaparte was a prominent French military and political leader who played a significant role during the French Revolution and Revolutionary Wars. He served as Emperor of the French from 1804 to 1814, with a brief return in 1815. Napoleon led France in the Napoleonic Wars against various coalitions, winning most battles and expanding his empire over continental Europe until its downfall in 1815. His military prowess and strategic campaigns are widely studied, and he remains a celebrated yet controversial figure in history.


In [34]:
summary6 = summarize_text(text_to_summarize, top_p=0.5)
print("Summary with low top_p:\n", summary6)

Summary with low top_p:
 Napoleon Bonaparte was a prominent French military and political leader who played a significant role during the French Revolution and Revolutionary Wars. He became Emperor of the French in 1804 and led successful campaigns against various coalitions in the Napoleonic Wars, expanding his empire across continental Europe. Known for his military prowess, Napoleon's legacy as a celebrated and controversial leader remains influential, with his wars and campaigns studied globally.


In [35]:
summary7 = summarize_text(text_to_summarize, frequency_penalty=1.0)
print("Summary with high frequency penalty:\n", summary7)

Summary with high frequency penalty:
 Napoleon Bonaparte was a prominent French military and political leader who played a significant role during the French Revolution and Revolutionary Wars. He became Emperor of the French in 1804, leading successful campaigns until his empire's collapse in 1815. Napoleon's dominance in European affairs during the Napoleonic Wars left a lasting legacy, with his military strategies and political influence being widely studied and debated globally.


In [36]:
summary8 = summarize_text(text_to_summarize, presence_penalty=1.0)
print("Summary with high presence penalty:\n", summary8)

Summary with high presence penalty:
 Napoleon Bonaparte was a French military and political leader who played a significant role during the French Revolution and Revolutionary Wars, becoming Emperor of the French in 1804. He led successful campaigns in various wars, dominating European affairs for over a decade and building a vast empire that ruled much of continental Europe. His military strategies are still studied globally, and he is considered one of history's greatest commanders, with a lasting legacy as a celebrated and controversial leader.


In [38]:
summary9 = summarize_text(text_to_summarize, logprobs=True)
print("Summary with logprobs parameter:\n", summary9)

Summary with logprobs parameter:
 Napoleon Bonaparte, a prominent French military and political figure, played a significant role during the French Revolution and Revolutionary Wars, becoming Emperor of the French in 1804. He led successful campaigns and dominated European affairs during the Napoleonic Wars, winning most battles and expanding his empire over continental Europe until its downfall in 1815. Considered one of the greatest commanders in history, his legacy is renowned and scrutinized worldwide for his military prowess and controversial leadership.


In [39]:
summary10 = summarize_text(text_to_summarize, logprobs=False)
print("Summary without logprobs parameter:\n", summary10)

Summary without logprobs parameter:
 Napoleon Bonaparte was a prominent French military and political leader who played a significant role during the French Revolution and the Revolutionary Wars. He became Emperor of the French in 1804 and led successful campaigns across Europe during the Napoleonic Wars, dominating global affairs for over a decade. Widely regarded as one of the greatest commanders in history, his legacy remains a subject of study and debate worldwide due to his lasting impact on European history.


Same parameters explanation as above

## 3. Translation
**Exercise:** Develop a tool that translates text from one language to another using the API.  
**Parameters to explore:** `temperature`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `echo`, `logit_bias`.

Comment what happen when you change the parameters 
(read documentation!)

In [47]:
def translate_spanish_to_french(text, temperature=0.7, max_tokens=150, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, logit_bias=None):
    
    client = OpenAI(
    api_key=openai.api_key,
    )
    
    messages = [
        {
            "role": "system",
            "content": "You are a translation assistant. Translate the following text from Spanish to French."
        },
        {
            "role": "user",
            "content": text
        }
    ]

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        logit_bias=logit_bias,
    )

    return chat_completion.choices[0].message.content

In [48]:
# Translation with default parameters
spanish_sentence = """El otoño es una estación mágica del año.\n"
    "Los árboles cambian de color, pasando del verde vibrante a tonos cálidos de rojo, naranja y amarillo.\n"
    "Las hojas caen suavemente al suelo, creando una alfombra crujiente bajo los pies.\n"
    "Es una época perfecta para dar largos paseos por el parque y disfrutar del aire fresco.\n"
    "Además, el otoño trae consigo la cosecha de manzanas y calabazas, llenando los mercados de sabores y aromas deliciosos."""

translation = translate_spanish_to_french(spanish_sentence)
print("Translation with default parameters:\n", translation)

# Additional translations with varied parameters
translation_high_temp = translate_spanish_to_french(spanish_sentence, temperature=1.0)
print("\nTranslation with high temperature:\n", translation_high_temp)

translation_low_temp = translate_spanish_to_french(spanish_sentence, temperature=0.3)
print("\nTranslation with low temperature:\n", translation_low_temp)

translation_short = translate_spanish_to_french(spanish_sentence, max_tokens=50)
print("\nTranslation with shorter max tokens:\n", translation_short)

translation_high_top_p = translate_spanish_to_french(spanish_sentence, top_p=0.5)
print("\nTranslation with low top_p:\n", translation_high_top_p)

translation_high_freq_penalty = translate_spanish_to_french(spanish_sentence, frequency_penalty=1.0)
print("\nTranslation with high frequency penalty:\n", translation_high_freq_penalty)

translation_high_presence_penalty = translate_spanish_to_french(spanish_sentence, presence_penalty=1.0)
print("\nTranslation with high presence penalty:\n", translation_high_presence_penalty)

logit_bias_example = {50256: -100}  # This is just an example, preventing the token 50256 from appearing
translation_logit_bias = translate_spanish_to_french(spanish_sentence, logit_bias=logit_bias_example)
print("\nTranslation with logit bias:\n", translation_logit_bias)


Translation with default parameters:
 L'automne est une saison magique de l'année.

Les arbres changent de couleur, passant du vert vibrant à des tons chauds de rouge, d'orange et de jaune.

Les feuilles tombent doucement au sol, créant un tapis croustillant sous les pieds.

C'est une période parfaite pour de longues promenades dans le parc et profiter de l'air frais.

De plus, l'automne apporte avec lui la récolte de pommes et de citrouilles, remplissant les marchés de saveurs et d'arômes délicieux.

Translation with high temperature:
 L'automne est une saison magique de l'année.

Les arbres changent de couleur, passant du vert vibrant à des tons chauds de rouge, d'orange et de jaune.

Les feuilles tombent doucement au sol, créant un tapis croustillant sous les pieds.

C'est un moment parfait pour de longues promenades dans le parc et profiter de l'air frais.

De plus, l'automne apporte la récolte des pommes et des citrouilles, remplissant les marchés de saveurs et d'arômes délicieux.

Same parameters explanation as above.

With the max token value we see the sentence gets cut in the middle.

Low temperature changes a few words

## 4. Sentiment Analysis
**Exercise:** Implement a sentiment analysis tool that determines the sentiment of a given text (positive, negative, neutral).  
**Parameters to explore:** `temperature`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `n`, `logprobs`.

Comment what happen when you change the parameters 
(read documentation!)

In [49]:
def analyze_sentiment(text, temperature=0.7, max_tokens=50, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, n=1, logprobs=None):
    
    client = OpenAI(
        api_key=openai.api_key,
    )
    
    messages = [
        {
            "role": "system",
            "content": "You are a sentiment analysis assistant. Analyze the sentiment of the following text and respond with 'positive', 'negative', or 'neutral'."
        },
        {
            "role": "user",
            "content": text
        }
    ]

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        n=n,
        logprobs=logprobs,
    )

    return chat_completion.choices[0].message.content

In [51]:
english_prompt = "I absolutely loved the new movie. It was fantastic and had great performances."

# Default parameters
sentiment_default = analyze_sentiment(english_prompt)
print("Sentiment with default parameters:\n", sentiment_default)

# High temperature (more creative)
sentiment_high_temp = analyze_sentiment(english_prompt, temperature=1.0)
print("\nSentiment with high temperature:\n", sentiment_high_temp)

# Low temperature (more deterministic)
sentiment_low_temp = analyze_sentiment(english_prompt, temperature=0.3)
print("\nSentiment with low temperature:\n", sentiment_low_temp)

# Shorter response
sentiment_short = analyze_sentiment(english_prompt, max_tokens=10)
print("\nSentiment with shorter max tokens:\n", sentiment_short)

# Lower top_p (more conservative)
sentiment_low_top_p = analyze_sentiment(english_prompt, top_p=0.5)
print("\nSentiment with low top_p:\n", sentiment_low_top_p)

# High frequency penalty (reduces repetition)
sentiment_high_freq_penalty = analyze_sentiment(english_prompt, frequency_penalty=1.0)
print("\nSentiment with high frequency penalty:\n", sentiment_high_freq_penalty)

# High presence penalty (encourages new words)
sentiment_high_presence_penalty = analyze_sentiment(english_prompt, presence_penalty=1.0)
print("\nSentiment with high presence penalty:\n", sentiment_high_presence_penalty)

# Multiple responses (n=3)
sentiments_multiple = analyze_sentiment(english_prompt, n=3)
print("\nSentiments with multiple responses:\n", sentiments_multiple)

# Logprobs enabled
sentiment_logprobs = analyze_sentiment(english_prompt, logprobs=True)
print("\nSentiment with logprobs as True:\n", sentiment_logprobs)

sentiment_logprobs = analyze_sentiment(english_prompt, logprobs=False)
print("\nSentiment with logprobs as False:\n", sentiment_logprobs)

Sentiment with default parameters:
 positive

Sentiment with high temperature:
 positive

Sentiment with low temperature:
 positive

Sentiment with shorter max tokens:
 positive

Sentiment with low top_p:
 positive

Sentiment with high frequency penalty:
 positive

Sentiment with high presence penalty:
 positive

Sentiments with multiple responses:
 positive

Sentiment with logprobs as True:
 positive

Sentiment with logprobs as False:
 positive


Here we see that since the sentence is easy to classify as positive, the reponses are all the same no matter the parameters.

In [52]:
# Now with a more nuanced sentence
english_prompt = "I enjoyed parts of the movie, but some scenes were too slow and the ending was disappointing."

# Default parameters
sentiment_default = analyze_sentiment(english_prompt)
print("Sentiment with default parameters:\n", sentiment_default)

# High temperature (more creative)
sentiment_high_temp = analyze_sentiment(english_prompt, temperature=1.0)
print("\nSentiment with high temperature:\n", sentiment_high_temp)

# Low temperature (more deterministic)
sentiment_low_temp = analyze_sentiment(english_prompt, temperature=0.3)
print("\nSentiment with low temperature:\n", sentiment_low_temp)

# Shorter response
sentiment_short = analyze_sentiment(english_prompt, max_tokens=10)
print("\nSentiment with shorter max tokens:\n", sentiment_short)

# Lower top_p (more conservative)
sentiment_low_top_p = analyze_sentiment(english_prompt, top_p=0.5)
print("\nSentiment with low top_p:\n", sentiment_low_top_p)

# High frequency penalty (reduces repetition)
sentiment_high_freq_penalty = analyze_sentiment(english_prompt, frequency_penalty=1.0)
print("\nSentiment with high frequency penalty:\n", sentiment_high_freq_penalty)

# High presence penalty (encourages new words)
sentiment_high_presence_penalty = analyze_sentiment(english_prompt, presence_penalty=1.0)
print("\nSentiment with high presence penalty:\n", sentiment_high_presence_penalty)

# Multiple responses (n=3)
sentiments_multiple = analyze_sentiment(english_prompt, n=3)
print("\nSentiments with multiple responses:\n", sentiments_multiple)

# Logprobs enabled
sentiment_logprobs = analyze_sentiment(english_prompt, logprobs=True)
print("\nSentiment with logprobs as True:\n", sentiment_logprobs)

sentiment_logprobs = analyze_sentiment(english_prompt, logprobs=False)
print("\nSentiment with logprobs as False:\n", sentiment_logprobs)

Sentiment with default parameters:
 negative

Sentiment with high temperature:
 mixed

Sentiment with low temperature:
 mixed

Sentiment with shorter max tokens:
 negative

Sentiment with low top_p:
 negative

Sentiment with high frequency penalty:
 negative

Sentiment with high presence penalty:
 mixed

Sentiments with multiple responses:
 negative

Sentiment with logprobs as True:
 negative

Sentiment with logprobs as False:
 negative


Interesting, even though we said to output neutral, positive or negative gpt returned mixed when he was unsure of the sentiment of the sentence.

When we input a high presence penalty, gpt returned mixed.

Low temperature and high temperature returned mixed as well.

We can clearly see differences in output by messing with the parameters.

It is then up to us to tune the parameters based on what behavior we want as output for a sentence that is biased.

## 5. Text Completion
**Exercise:** Create a text completion application that generates text based on an initial prompt.  
**Parameters to explore:** `temperature`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `stop`, `best_of`.

Comment what happen when you change the parameters 
(read documentation!)

In [55]:
def complete_text(prompt, temperature=0.7, max_tokens=100, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=None):
    
    client = OpenAI(
        api_key=openai.api_key,
    )
    
    messages = [
        {
            "role": "system",
            "content": "You are a creative writing assistant. Continue the following text."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop,
    )

    return chat_completion.choices[0].message.content

In [56]:
initial_prompt = "Once upon a time in a distant land, there was a small village surrounded by mountains."

# Default parameters
completion_default = complete_text(initial_prompt)
print("Completion with default parameters:\n", completion_default)

# High temperature (more creative)
completion_high_temp = complete_text(initial_prompt, temperature=1.0)
print("\nCompletion with high temperature:\n", completion_high_temp)

# Low temperature (more deterministic)
completion_low_temp = complete_text(initial_prompt, temperature=0.3)
print("\nCompletion with low temperature:\n", completion_low_temp)

# Shorter response
completion_short = complete_text(initial_prompt, max_tokens=50)
print("\nCompletion with shorter max tokens:\n", completion_short)

# Lower top_p (more conservative)
completion_low_top_p = complete_text(initial_prompt, top_p=0.5)
print("\nCompletion with low top_p:\n", completion_low_top_p)

# High frequency penalty (reduces repetition)
completion_high_freq_penalty = complete_text(initial_prompt, frequency_penalty=1.0)
print("\nCompletion with high frequency penalty:\n", completion_high_freq_penalty)

# High presence penalty (encourages new words)
completion_high_presence_penalty = complete_text(initial_prompt, presence_penalty=1.0)
print("\nCompletion with high presence penalty:\n", completion_high_presence_penalty)

# Using stop sequence
completion_with_stop = complete_text(initial_prompt, stop=["."])
print("\nCompletion with stop sequence:\n", completion_with_stop)

Completion with default parameters:
 The villagers lived simple lives, relying on the land for their food and water. Each morning, they would wake up to the sound of birds chirping and the gentle rustling of the wind through the trees. Life in the village was peaceful and harmonious, with everyone working together to ensure the well-being of the community.

One day, a mysterious traveler arrived in the village. He was tall and cloaked in a long, dark robe, with a hood that obscured his face. The villagers were

Completion with high temperature:
 The village was known for its lush green fields, crystal-clear streams, and vibrant wildlife. The villagers lived in harmony with nature, tending to their crops and animals with great care. The mountain range that loomed high above the village was said to be home to ancient spirits that watched over the land. Legends spoke of hidden caves filled with treasures and magical artifacts waiting to be discovered by those brave enough to seek them out

This time we get very varied answers based on the change in parameters.

# BONUS: Google Vertex AI

## 1. Basic Conversation
**Exercise:** Create a basic chatbot using Google Vertex AI to answer questions about a given topic.  
**Parameters to explore:** `temperature`, `max_output_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `n`, `stop`.

Comment what happen when you change the parameters 
(read documentation!)

## 2. Summarization
**Exercise:** Develop a script that summarizes long text inputs using Google Vertex AI.  
**Parameters to explore:** `temperature`, `max_output_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `best_of`, `logprobs`.

Comment what happen when you change the parameters 
(read documentation!)

## 3. Translation
**Exercise:** Create a tool that translates text from one language to another using Google Vertex AI.  
**Parameters to explore:** `temperature`, `max_output_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `echo`, `logit_bias`.

Comment what happen when you change the parameters 
(read documentation!)

## 4. Sentiment Analysis
**Exercise:** Implement a sentiment analysis tool using Google Vertex AI to determine the sentiment of a given text.  
**Parameters to explore:** `temperature`, `max_output_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `n`, `logprobs`.

Comment what happen when you change the parameters 
(read documentation!)

## 5. Text Completion
**Exercise:** Develop a text completion application using Google Vertex AI to generate text based on an initial prompt.  
**Parameters to explore:** `temperature`, `max_output_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `stop`, `best_of`.

Comment what happen when you change the parameters 
(read documentation!)